In [1]:
%matplotlib inline
import os
import shutil
import sys
import numpy as np
import pandas as pd
import mxnet as mx
from mxnet import gluon
from mxnet import image
from mxnet import nd
from mxnet import init
from mxnet import autograd
from mxnet.gluon.data import vision
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models
from PIL import Image
from time import time
from tqdm import tqdm
import h5py
from sklearn.model_selection import train_test_split

In [2]:
def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter)
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

In [3]:
with h5py.File('features.h5', 'r') as f:
    features_vgg_train = np.array(f['vgg'])
    features_resnet_train = np.array(f['resnet'])
    features_densenet_train = np.array(f['densenet'])
    features_inception_train = np.array(f['inception'])
    labels_train = np.array(f['labels'])

In [4]:
with h5py.File('features_test.h5', 'r') as f:
    features_vgg_test = np.array(f['vgg'])
    features_resnet_test = np.array(f['resnet'])
    features_densenet_test = np.array(f['densenet'])
    features_inception_test = np.array(f['inception'])

In [5]:
features_resnet_train = features_resnet_train.reshape(features_resnet_train.shape[:2])
features_inception_train = features_inception_train.reshape(features_inception_train.shape[:2])

features_train = np.concatenate([features_resnet_train, features_densenet_train, features_inception_train], axis=-1)

In [6]:
X_train, X_val, y_train, y_val = train_test_split(features_train, labels_train, test_size=0.1)

# dataset_train = gluon.data.ArrayDataset(nd.array(X_train), nd.array(y_train))
dataset_train = gluon.data.ArrayDataset(nd.array(features_train), nd.array(labels_train))
dataset_val = gluon.data.ArrayDataset(nd.array(X_val), nd.array(y_val))

batch_size = 256
data_iter_train = gluon.data.DataLoader(dataset_train, batch_size, shuffle=True)
data_iter_val = gluon.data.DataLoader(dataset_val, batch_size)

In [7]:
X_train.shape

(27721, 6016)

In [8]:
ctx = mx.gpu(0)
net = nn.Sequential()
with net.name_scope():
    net.add(nn.BatchNorm())
    net.add(nn.Dense(1024))
    net.add(nn.BatchNorm())
    net.add(nn.Activation('relu'))
    net.add(nn.Dropout(0.5))
    net.add(nn.Dense(120))

net.initialize(ctx=ctx)
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
lr_sch = mx.lr_scheduler.FactorScheduler(step=1500, factor=0.5, stop_factor_lr=1e-11)
trainer = gluon.Trainer(net.collect_params(), 'adam', 
                        {'learning_rate': 1e-3, 'lr_scheduler': lr_sch})
net

Sequential(
  (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, in_channels=None)
  (1): Dense(None -> 1024, linear)
  (2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, in_channels=None)
  (3): Activation(relu)
  (4): Dropout(p = 0.5)
  (5): Dense(None -> 120, linear)
)

In [9]:
from time import time
for epoch in range(100):
    start = time()
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train)
    for data, label in data_iter_train:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        
        loss.backward()
        trainer.step(batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)
    
    val_loss, val_acc = evaluate(net, data_iter_val)
    end = time() - start
    print("Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%%, times %.2fs, lr %s" % (
        epoch+1, train_loss/steps, train_acc/steps*100, val_loss, val_acc*100, end, str(trainer.learning_rate)))

Epoch 1. loss: 0.5446, acc: 85.47%, val_loss 0.1422, val_acc 95.31%, times 1.89s, lr 0.001
Epoch 2. loss: 0.1710, acc: 94.67%, val_loss 0.0769, val_acc 97.93%, times 1.56s, lr 0.001
Epoch 3. loss: 0.1104, acc: 96.50%, val_loss 0.0539, val_acc 98.53%, times 1.65s, lr 0.001
Epoch 4. loss: 0.0783, acc: 97.49%, val_loss 0.0346, val_acc 99.10%, times 1.75s, lr 0.001
Epoch 5. loss: 0.0575, acc: 98.26%, val_loss 0.0212, val_acc 99.49%, times 1.73s, lr 0.001
Epoch 6. loss: 0.0458, acc: 98.65%, val_loss 0.0196, val_acc 99.40%, times 1.67s, lr 0.001
Epoch 7. loss: 0.0385, acc: 98.87%, val_loss 0.0283, val_acc 99.22%, times 1.65s, lr 0.001
Epoch 8. loss: 0.0376, acc: 98.90%, val_loss 0.0205, val_acc 99.37%, times 1.67s, lr 0.001
Epoch 9. loss: 0.0336, acc: 99.00%, val_loss 0.0257, val_acc 99.22%, times 1.66s, lr 0.001
Epoch 10. loss: 0.0312, acc: 99.16%, val_loss 0.0132, val_acc 99.70%, times 1.68s, lr 0.001
Epoch 11. loss: 0.0278, acc: 99.24%, val_loss 0.0090, val_acc 99.79%, times 1.63s, lr 0.0

Epoch 88. loss: 0.0040, acc: 99.81%, val_loss 0.0013, val_acc 99.91%, times 1.64s, lr 7.8125e-06
Epoch 89. loss: 0.0035, acc: 99.85%, val_loss 0.0014, val_acc 99.91%, times 1.70s, lr 7.8125e-06
Epoch 90. loss: 0.0036, acc: 99.84%, val_loss 0.0013, val_acc 99.91%, times 1.77s, lr 7.8125e-06
Epoch 91. loss: 0.0030, acc: 99.88%, val_loss 0.0013, val_acc 99.91%, times 1.58s, lr 7.8125e-06
Epoch 92. loss: 0.0042, acc: 99.84%, val_loss 0.0014, val_acc 99.91%, times 1.45s, lr 7.8125e-06
Epoch 93. loss: 0.0043, acc: 99.81%, val_loss 0.0014, val_acc 99.91%, times 1.67s, lr 7.8125e-06
Epoch 94. loss: 0.0042, acc: 99.82%, val_loss 0.0014, val_acc 99.91%, times 1.65s, lr 7.8125e-06
Epoch 95. loss: 0.0033, acc: 99.87%, val_loss 0.0014, val_acc 99.91%, times 1.68s, lr 7.8125e-06
Epoch 96. loss: 0.0036, acc: 99.83%, val_loss 0.0015, val_acc 99.88%, times 1.70s, lr 7.8125e-06
Epoch 97. loss: 0.0038, acc: 99.84%, val_loss 0.0015, val_acc 99.88%, times 1.71s, lr 7.8125e-06
Epoch 98. loss: 0.0037, acc: 9

In [10]:
features_resnet_test = features_resnet_test.reshape(features_resnet_test.shape[:2])
features_inception_test = features_inception_test.reshape(features_inception_test.shape[:2])

features_test = np.concatenate([features_resnet_test, features_densenet_test, features_inception_test], axis=-1)

In [11]:
output = nd.softmax(net(nd.array(features_test).as_in_context(ctx))).asnumpy()

In [12]:
df = pd.read_csv('sample_submission.csv')

for i, c in enumerate(df.columns[1:]):
    df[c] = output[:,i]

df.to_csv('pred.csv', index=None)

In [13]:
df2 = df.copy()

In [15]:
output.argmax(axis=1)

array([61, 94, 40, ..., 36, 70, 59])

In [24]:
labels = df.columns[1:]

In [25]:
def mkdir_if_not_exist(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))

In [34]:
for i, l in zip(df['id'], output.argmax(axis=1)):
    if not os.path.exists(os.path.join('/home/samael/kaggle/dogs/data2/train2', labels[l])):
        os.makedirs(os.path.join('/home/samael/kaggle/dogs/data2/train2', labels[l]))
    shutil.copy('/home/samael/kaggle/dogs/data2/test/unknown/' + i + '.jpg', os.path.join('/home/samael/kaggle/dogs/data2/train2', labels[l]))